In [1]:
# import functions from modules
import sys
import os

# Get absolute path to src directory
src_path = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'src'))

# Only add to path if not already there
if src_path not in sys.path:
    sys.path.append(src_path)

from portopt.utils import write_table
from portopt.config import load_config
# holdings_fact
from portopt.holdings import load_and_consolidate_holdings
# price_fact
from portopt.market_data import get_latest_ticker_prices
# factor_dim
from portopt.factor import load_factor_dimension
# factor_weights
from portopt.factor import load_factor_weights

# duckdb
import duckdb

In [ ]:
# holdings directory
portfolio_dir = "../../data/portfolio"
holdings_dir = os.path.join(portfolio_dir, "holdings")

print("portfolio_dir:", portfolio_dir)
print("holdings_dir:", holdings_dir)

In [7]:
# load config
config_file = os.path.join(portfolio_dir, "config.yml")
config = load_config(config_file)

In [34]:
column_formats = {
    'Ticker': {'width': 14},
    'Level_0': {'width': 14},
    'Level_1': {'width': 14},
    'Level_2': {'width': 14},
    'Level_3': {'width': 14},
    'Level_4': {'width': 14},
    'Level_5': {'width': 14},
    'Level_6': {'width': 14},
    'Factor': {'width': 14},
    'Original Ticker': {'width': 14},
    'Asset Class': {'width': 25},
    'Account': {'width': 15, 'align': '<'},
    'Account Number': {'width': 14, 'align': '^'},
    'Account Name': {'width': 15, 'align': '<'},
    'Quantity': {'width': 10, 'decimal': 3},
    'Original Quantity': {'width': 10, 'decimal': 3},
    'Cost Basis': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Price': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Total Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Original Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'Weight': {'width': 16, 'decimal': 2, 'type':'%'}
}

In [ ]:
# load the dimensions and facts
# holdings_fact
holdings_fact = load_and_consolidate_holdings(holdings_dir,
                                              config=config,
                                              verbose=False)

write_table(holdings_fact, columns=column_formats)

In [ ]:
# price_fact
tickers = holdings_fact.index.get_level_values('Ticker').unique()
price_fact = get_latest_ticker_prices(tickers, verbose=True)

write_table(price_fact, columns=column_formats)

In [ ]:
# factor_dim
factor_dim = load_factor_dimension(config)
write_table(factor_dim, columns=column_formats)


In [ ]:
# factor_weights
file_path = os.path.join(portfolio_dir, "asset_class_weights_matrix.csv")
factor_weights = load_factor_weights(file_path, factor_dim)
write_table(factor_weights, columns=column_formats)

In [ ]:
import duckdb

# Connect to an in-memory DuckDB instance
con = duckdb.connect()

holdings_fact = holdings_fact.reset_index()
price_fact = price_fact.reset_index()
factor_dim = factor_dim.reset_index()
factor_weights = factor_weights.reset_index()

# Register DataFrames as DuckDB tables
con.register("holdings_fact", holdings_fact)
con.register("price_fact", price_fact)
con.register("factor_dim", factor_dim)
con.register("factor_weights", factor_weights)

In [ ]:
query = """
WITH
base_with_weights AS (
  SELECT
    h.account,
    h.ticker,
    h.quantity,
    p.price,
    f.level_0,
    f.level_1,
    SUM(fw.weight) AS weight,
  FROM
    holdings_fact h
    LEFT JOIN price_fact p
      ON h.ticker = p.ticker
    LEFT JOIN factor_weights fw
      ON h.ticker = fw.ticker
    LEFT JOIN factor_dim f
      ON fw.factor = f.factor
  GROUP BY h.account, h.ticker, h.quantity, p.price, f.level_0, f.level_1
),
filtered_position_values AS (
  SELECT
    account,
    ticker,
    (quantity * price * weight) as Value
  FROM base_with_weights
  WHERE level_0 = 'Equity' AND level_1 = 'US'
  GROUP BY account, ticker, quantity, price, weight
)
SELECT
  account,
  SUM(Value) AS Value
FROM filtered_position_values
GROUP BY account
"""
result = con.execute(query).fetchdf()
write_table(result, columns=column_formats)

In [ ]:
query = """
SELECT
  h.account,
  SUM(h.quantity * p.price) AS Value,
  (Value / (
       SELECT 
         SUM(h.quantity * p.price)
         FROM holdings_fact h
         JOIN price_fact p ON h.ticker = p.ticker)
  ) AS Allocation
FROM
  holdings_fact h
  LEFT JOIN price_fact p
    ON h.ticker = p.ticker
  LEFT JOIN factor_weights fw
    ON h.ticker = fw.ticker
  LEFT JOIN factor_dim f
    ON fw.factor = f.factor
WHERE
  f.level_0 = 'Equity' AND f.level_1 = 'US'
GROUP BY h.account
"""
result = con.execute(query).fetchdf()
write_table(result, columns=column_formats)

In [ ]:
query = """
    WITH ticker_values AS (
        SELECT 
            h.Ticker,
            SUM(h.Quantity * p.Price) AS ticker_total_value
        FROM holdings_fact h
        JOIN price_fact p ON h.Ticker = p.Ticker
        GROUP BY h.Ticker
    ),
    portfolio_total AS (
        SELECT SUM(ticker_total_value) AS total_portfolio_value
        FROM ticker_values
    )
    SELECT 
        tv.Ticker,
        tv.ticker_total_value as "Total Value",
        (tv.ticker_total_value / pt.total_portfolio_value) AS "Allocation"
    FROM ticker_values tv
    CROSS JOIN portfolio_total pt
    ORDER BY ticker_total_value DESC;
"""

result = con.execute(query).fetchdf()
write_table(result, columns=column_formats)